In [1]:
import warnings
#importing libraries for warnings
warnings.filterwarnings("ignore")
import pandas as pd
#importing libraries for pandas
import numpy as np
#importing libraries for numpy
import statsmodels.api as sm
#importing libraries for statsmodels
import scipy
#importing libraries for scipy
import scipy.stats as stats
#importing libraries for scipy
from matplotlib import pyplot as plt
#importing libraries for matplotlib
import seaborn  as sns
#importing libraries for seaborn
from plotly import express as px
#importing libraries for seaborn
import os
#importing os

In [2]:
import nltk
#importing nltk
nltk.download('stopwords')
#downloading stop words

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\A2G\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\stopwords.zip.


True

In [2]:
# pip install nltk

In [3]:
import nltk
#importing nltk
nltk.download('punkt')
#downloading punkt

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\A2G\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [4]:
from nltk.corpus import stopwords
#importing libraries for stopwords
from nltk.stem import PorterStemmer
#importing libraries for PorterStemmer
from nltk.tokenize import word_tokenize
#importing libraries for word_tokenize
text1 = 'This is an example sentence.'
#initialize texts
stop_wordss = set(stopwords.words('english'))
stemmer = PorterStemmer()
token1s = word_tokenize(text1.lower())
filtered_tokens = [stemmer.stem(token) for token in token1s if token not in stop_wordss]

In [5]:
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.tokenize import word_tokenize
text1 = 'This is an example sentence.'
stop_words1 = set(stopwords.words('english'))
#defining text
stemmers = PorterStemmer()
tokens = word_tokenize(text1.lower())
filtered_tokens = [stemmer.stem(token) for token in tokens if token not in stop_words1]

In [1]:
pip install --upgrade whoosh

Note: you may need to restart the kernel to use updated packages.


In [10]:
from whoosh import fields
from whoosh.index import create_in
#importing whoosh libraries
schemas = fields.Schema(
    title=fields.TEXT(stored=True),
    authors=fields.KEYWORD(stored=True),
    year=fields.NUMERIC(stored=True),
    url=fields.ID(stored=True),
    abstract=fields.TEXT(stored=True)
)
#intialize index
index_dir = 'path/to/index/dir'
if not os.path.exists(index_dir):
    os.makedirs(index_dir)
indexs = create_in(index_dir, schemas)

In [12]:
import requests
from bs4 import BeautifulSoup
#importing libraries of bs4
def extract_metadata(url):
    """
    Extract metadata for a publication from its web page.
    """
    page = requests.get(url)
    soup = BeautifulSoup(page.content, 'html.parser')
    #defining variables 
    title = soup.find('h1', {'class': 'title'}).text.strip()
    authors = [a.text.strip() for a in soup.find_all('a', {'class': 'person-name'})]
    year = int(soup.find('span', {'class': 'year'}).text.strip())
    abstract = soup.find('div', {'class': 'abstract'}).text.strip()
    pureportal_url = soup.find('a', {'class': 'pure-link'})['href']
    #defining metadata
    metadata = {
        'title': title,
        'authors': authors,
        'year': year,
        'url': url,
        'abstract': abstract,
        'pureportal_url': pureportal_url
    }

    return metadata

In [2]:
import os
import requests
from bs4 import BeautifulSoup
from whoosh import fields
from whoosh.index import create_in
from whoosh.qparser import MultifieldParser

# Defining the queries based on schema for the particular search index
schema = fields.Schema(
    title=fields.TEXT(stored=True),
    authors=fields.KEYWORD(stored=True),
    year=fields.NUMERIC(stored=True),
    url=fields.ID(stored=True),
    abstract=fields.TEXT(stored=True),
    pureportal_url=fields.ID(stored=True)
)
# Creating of the taegeted search index 
index_dirs = 'path/to/index/dir'
if not os.path.exists(index_dirs):
    os.makedirs(index_dirs)
index = create_in(index_dirs, schema)
# Defining URL based on CSM publications
base_urls = 'https://pureportal.coventry.ac.uk'
# Defining URL based on CSM publications search page
search_urls = 'https://www.coventry.ac.uk/research/areas-of-research/computational-science-mathematical-modelling/'
# Defining the major number of particular pages to crawl
max_pages = 5
# Crawling based on the search results pages and additon of publications based on search index
for page_num in range(1, max_pages + 1):
    # Fetching data from the search results page
    page_urls = search_urls.format(page_num)
    page = requests.get(page_urls)
    soup = BeautifulSoup(page.content, 'html.parser')
    #  publication links based on the url link
    publication_links = soup.find_all('a', {'class': 'link-item'})
    # Extracting metadata for each publication and add it to the search index
    for link in publication_links:
        publication_url = base_urls + link['href']
        metadata = extract_metadata(publication_url)
        with index.writer() as writer:
            writer.add_document(title=metadata['title'],
                                authors=metadata['authors'],
                                year=metadata['year'],
                                url=metadata['url'],
                                abstract=metadata['abstract'],
                                pureportal_url=metadata['pureportal_url'])

In [4]:
def extract_metadata(url):
    # exraction of the data from the publication details page
    page = requests.get(url)
    soup = BeautifulSoup(page.content, 'html.parser')
    # Extracting targeted metadata from the following page
    title = soup.find('h1', {'class': 'title'}).text.strip()
    authors = [a.text.strip() for a in soup.find_all('span', {'itemprop': 'author'})]
    year = int(soup.find('span', {'itemprop': 'datePublished'}).text.strip())
    abstract = soup.find('div', {'class': 'row abstract'}).find('div', {'class': 'value'}).text.strip()
    pureportal_url = url
    # genarytion of the metadata for dictionary
    metadata = {
        'title': title,
        'authors': authors,
        'year': year,
        'url': url,
        'abstract': abstract,
        'pureportal_url': pureportal_url
    }
    #returning metadata
    return metadata

In [16]:
# pip install metadata

In [5]:
    # Extracting targeted metadata for individual publication and additon of the search index
for link in publication_links:
     publication_url = base_urls + link['href']
     #defining metadata
     metadata = extract_metadata(publication_url)
     # Adding another metadata of search index
     with index.writer() as writer:
        writer.add_document(**metadata)

In [6]:
# extraction of metadata based on the publication with the details page
def extract_metadata(url):
    # Fetching values of the publication based ondetails page
    page = requests.get(url)
    soup = BeautifulSoup(page.content, 'html.parser')
    # Extracting details of metadata from the page
    title = soup.find('h1', {'class': 'title'}).text.strip()
    authors = [a.text.strip() for a in soup.find_all('span', {'itemprop': 'author'})]
    year = int(soup.find('span', {'itemprop': 'datePublished'}).text.strip())
    abstract = soup.find('div', {'class': 'row abstract'}).find('div', {'class': 'value'}).text.strip()
    pureportal_url = url

In [7]:
# Crawling based on the results pages and additon of the publications based on the search index
for page_num in range(1, max_pages + 1):
    page_url = search_urls.format(page_num)
    page = requests.get(page_url)
    soup = BeautifulSoup(page.content, 'html.parser')
    publication_links = soup.find_all('a', {'class': 'link-item'})
    for link in publication_links:
        publication_url = base_urls + link['https://scholar.google.com/scholar?hl=en&as_sdt=0%2C5&q=information+technology&btnG=&oq=informa']
        metadata = extract_metadata(publication_url)
        with index.writer() as writer:
            writer.add_document(**metadata)


In [8]:
# Define  parser query

In [9]:
query_parsers = MultifieldParser(['title', 'authors', 'abstract'], schema=index.schema)

query_strs = input('Enter your search query: ')

query1 = query_parsers.parse(query_strs)
with index.searcher() as searcher:
    result1s = searcher.search(query1)
    print(f'Search results ({len(result1s)} total):')
    for i, result in enumerate(result1s):
        print(f'{i + 1}. {result["title"]} ({result["year"]}) by {", ".join(result["authors"])}')
        print(f'   Abstract: {result["abstract"]}')
        print(f'   URL: {result["url"]}\n')

Search results (0 total):
